In [ ]:
!pip install -q streamlit

     |████████████████████████████████| 7.2MB 2.6MB/s 
     |████████████████████████████████| 4.4MB 34.2MB/s 
     |████████████████████████████████| 112kB 46.8MB/s 
     |████████████████████████████████| 102kB 9.6MB/s 
     |████████████████████████████████| 122kB 41.2MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.4 which is incompatible.


In [ ]:
!pip install pyngrok

  Created wheel for pyngrok: filename=pyngrok-4.1.10-cp36-none-any.whl size=16377 sha256=93f68cb4513c04050e3894060dff58b586707c3c7a7afd19e43499b273e71cfe
  Stored in directory: /root/.cache/pip/wheels/0d/7f/06/e8079dd6f80eb6683fac47c4e1cb720b61cd82d6e4c13d87a5
Successfully built pyngrok


In [ ]:
! ngrok authtoken xxxxxxxxxxxxxxxxxxxxxx

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/Colab Notebooks/Image_Classification_TransferLearning/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%%writefile score.py

import tensorflow as tf
import numpy as np
import streamlit as st
from PIL import Image
import requests
from io import BytesIO

st.set_option('deprecation.showfileUploaderEncoding', False)
st.title('Bean Image Classifier')
st.text('Provide URL of bean Image for image classification')

@st.cache(allow_output_mutation=True)
def load_model():
  model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/Image_Classification_TransferLearning/')
  return model

with st.spinner('Loading Model Into Mmory....'):
  model = load_model()

Classes = ['angulat_leaf_spot', 'bean_rust', 'healthy']

def scale(image):
  image = tf.cast(image, tf.float32)
  image /= 255.0

  return tf.image.resize(image, [224, 224])

def decode_img(img):
  img = tf.image.decode_jpeg(img, channels=3)
  img = scale(img)
  return np.expand_dims(img, axis=0)

path = st.text_input('Enter Image URL to Classify.. ', 
                     'http://barmac.com.au/wp-content/uploads/sites/3/2016/01/Angular-Leaf-Spot-Beans1.jpg')
if path is not None:
  content = requests.get(path).content
  st.write("Predicted class")
  with st.spinner('Classifying...'):
    label = np.argmax(model.predict(decode_img(content)), axis=1)
    st.write(Classes[label[0]])
  st.write("")

  image = Image.open(BytesIO(content))
  st.image(image, caption='Classifying Bean Image', use_column_width=True)

Writing score.py


In [ ]:
!nohup streamlit run score.py &

nohup: appending output to 'nohup.out'


In [ ]:
from pyngrok import ngrok
url = ngrok.connect(port=8501)
url

'http://14444b1854cf.ngrok.io'